In [1]:
#압축파일 풀기
import zipfile
import shutil
import os
folder_path = "./tagging_data/main_annotated_data/"

zip_ = zipfile.ZipFile(os.path.join(folder_path, "tagtog_League_of_Legend_Relation_Extraction.zip"))
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
zip_.extractall(folder_path)

In [2]:
import json
import glob
import re
import os

pilot_or_main = 'main'
#폴더 경로
folder_name = os.path.join(folder_path, 'League_of_Legend_Relation_Extraction/')

#context list
context_name_list = os.listdir(folder_name + "ann.json/master/pool/"+pilot_or_main)

#relation 폴더 경로
relation_folder_paths = [folder_name + "ann.json/master/pool/"+pilot_or_main + "/"+c for c in os.listdir(folder_name + "ann.json/master/pool/"+pilot_or_main)]

#context 폴더 경로
# contexts_folders_paths = glob.glob(folder_name + "plain.html/pool/*")
contexts_folders_paths = [folder_name + "plain.html/pool/"+pilot_or_main + "/"+c for c in context_name_list]


#anntation_lenged 정보
annotation_legend = folder_name + "annotations-legend.json"
with open(annotation_legend,"r") as f:
    annotation_legend = json.load(f)
    

In [3]:
context_name_list

['T2196', 'T2077', 'T2046', 'T2258']

In [4]:
relation_folder_paths

['./tagging_data/main_annotated_data/League_of_Legend_Relation_Extraction/ann.json/master/pool/main/T2196',
 './tagging_data/main_annotated_data/League_of_Legend_Relation_Extraction/ann.json/master/pool/main/T2077',
 './tagging_data/main_annotated_data/League_of_Legend_Relation_Extraction/ann.json/master/pool/main/T2046',
 './tagging_data/main_annotated_data/League_of_Legend_Relation_Extraction/ann.json/master/pool/main/T2258']

In [5]:
contexts_folders_paths

['./tagging_data/main_annotated_data/League_of_Legend_Relation_Extraction/plain.html/pool/main/T2196',
 './tagging_data/main_annotated_data/League_of_Legend_Relation_Extraction/plain.html/pool/main/T2077',
 './tagging_data/main_annotated_data/League_of_Legend_Relation_Extraction/plain.html/pool/main/T2046',
 './tagging_data/main_annotated_data/League_of_Legend_Relation_Extraction/plain.html/pool/main/T2258']

In [6]:
#relation dictionary 파일로부터 subject와 object의 entity 정보를 추출해주는 함수
def get_needed_relation_data(tmp_relation):
    subject_token = re.findall("\(+(.+)+\)",annotation_legend[tmp_relation["relations"][0]['classId']])[0].split("|")[0]
    if subject_token == tmp_relation['entities'][0]['classId']:
        sub_entity, obj_entity = tmp_relation['entities']
    else:
        obj_entity, sub_entity  = tmp_relation['entities']
    
    # get preprocessed entities
    def _get_entity(entity):
        outputs = entity['offsets'][0]
        outputs['type'] = annotation_legend[entity['classId']].split("-")[1].lower()
        return outputs
    
    output_subject = _get_entity(sub_entity)
    output_object = _get_entity(obj_entity)
    return output_subject, output_object

In [7]:
#relation dictionary 파일로부터 title(relation)을 추출해주는 함수
def get_label(relation_json):
    label_tag = relation_json['relations'][0]['classId'] #r_6
    try:
        _,sub_type, label = annotation_legend[re.findall("\(+(.+)+\|",annotation_legend[label_tag])[0]].split("-")
        return f"{sub_type}:{label}"
    except:
        _,sub_type, = annotation_legend[re.findall("\(+(.+)+\|",annotation_legend[label_tag])[0]].split("-")
        return f"{sub_type}:no_relation"

In [8]:
#html 파일로부터 text만 추출해주는 함수
def get_context_from_html(html_file):
    html_file = re.sub(r"\n","", html_file)
    return re.findall("(<pre.+>)(.+)(</pre>)",html_file)[0][1]

In [9]:
#entity 정보가 포함된 sentence를 생성해주는 함수
def get_sentence_with_entites(subject_entity, object_entity, sentence):
    if subject_entity['start'] < object_entity['start']:
        entity1,entity2 = subject_entity, object_entity
    else:
        entity1,entity2 = object_entity, subject_entity
    
    #entity 시작 위치 및 길이 
    ett1_stt, ett1_len = entity1['start'], len(entity1['text'])
    ett2_stt, ett2_len = entity2['start'], len(entity2['text'])
    
    #문장 분리
    bf, ett1, mid, ett2, af = sentence[:ett1_stt], \
                            sentence[ett1_stt:ett1_stt+ett1_len], \
                            sentence[ett1_stt+ett1_len:ett2_stt], \
                            sentence[ett2_stt:ett2_stt+ett2_len], \
                            sentence[ett2_stt+ett2_len:]

    
    if subject_entity['start'] < object_entity['start']:
        ett1,ett2 = f"<sbj:{ett1}>", f"<obj:{ett2}>"
    else:
        ett1,ett2 = f"<obj:{ett1}>", f"<sbj:{ett2}>"
    
    return "".join([bf, ett1, mid, ett2, af])

In [10]:
#dataframe column
# id : context title(e.g : 카카오게임, 11번가 등)
# sentence w/o entity
# sentence w entity
# subject_entity
# object_entity
# class

id_list = []
sentence_list = []
sentence_with_entities_list = []
subject_entity_list = []
object_entity_list = []
relation_list = []

# tagtog 데이터를 CSV 형태로 변경
for context_name, relation_folder, contexts_folder in zip(context_name_list, relation_folder_paths, contexts_folders_paths):
    # relation files와 context files 리스트 출력
    file_ids = [file_name.split(".ann.")[0] for file_name in os.listdir(relation_folder)]
    file_nums = [ids.split("-")[1] for ids in file_ids]
    relation_files = [relation_folder + "/"+ file_id + ".ann.json" for file_id in file_ids]
    context_files = [contexts_folder + "/"+ file_id + ".plain.html" for file_id in file_ids]
    
    
    
    #json으로 된 relation data와 html로 된 context 데이터 읽기
    for relation_file, context_file, file_num in zip(relation_files,context_files, file_nums):
        
        #subject, object 정보 추출
        with open(relation_file, "r") as f:
            relation_json = json.load(f)
        try:
            tmp_subject, tmp_object = get_needed_relation_data(relation_json) #subject, object
            tmp_label = get_label(relation_json)
        except:
            continue
        
        #sentence, sentence with entities 정보 추출
        with open(context_file, "r") as f:
            context_json = f.read()
        tmp_sentence = get_context_from_html(context_json)
        tmp_sentence_w_entities = get_sentence_with_entites(tmp_subject,tmp_object,tmp_sentence)
        
        #각 list에 데이터 저장
        id_list.append(f"{context_name}_{file_num}")
        sentence_list.append(tmp_sentence)
        sentence_with_entities_list.append(tmp_sentence_w_entities)
        subject_entity_list.append(tmp_subject)
        object_entity_list.append(tmp_object)
        relation_list.append(tmp_label.lower())
    

In [11]:
len(id_list)

452

In [15]:
for i in range(len(relation_list)):
    if "관계없음" in relation_list[i] or "관계_없음" in relation_list[i]:
        relation_list[i] = "관계없음"

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split

data_dict = {
    "id" : range(len(id_list)), 
    "sentence" : sentence_list, 
    "subject_entity" : subject_entity_list, 
    "object_entity" : object_entity_list, 
    "label" : relation_list, 
    "source" : "wikipedia"
}


data_df = pd.DataFrame(data_dict, columns=data_dict.keys())

In [17]:
data_df

,id,sentence,subject_entity,object_entity,label,source
0,0,"그 누구도 다른 사람의 과거에 관심이 없기 때문에 법의 심판, 빚, 박해를 피해 도...","{'start': 9, 'text': '사람', 'type': 'per'}","{'start': 50, 'text': '사람', 'type': 'per'}",관계없음,wikipedia
1,1,소나는 아군의 상처를 어루만지기 위해서 뿐 아니라 방심하고 있는 적을 공격하기 위해...,"{'start': 4, 'text': '아군', 'type': 'org'}","{'start': 0, 'text': '소나', 'type': 'per'}",org:구성원,wikipedia
2,2,"오래 전에 시각을 잃었으나, 고향 땅의 신성한 균형을 해치려 드는 자를 막아내는 데...","{'start': 16, 'text': '고향', 'type': 'org'}","{'start': 37, 'text': '자', 'type': 'per'}",관계없음,wikipedia
3,3,또한 보수를 받고 일을 하고 무법의 황무지로 돌아가는 용병들도 있다.,"{'start': 16, 'text': '무법의 황무지', 'type': 'org'}","{'start': 30, 'text': '용병', 'type': 'org'}",org:구성원,wikipedia
4,4,푸른 불꽃 제도에 위치한 빌지워터는 다른 항구 도시와는 다르다.,"{'start': 0, 'text': '푸른 불꽃 제도', 'type': 'org'}","{'start': 14, 'text': '빌지워터', 'type': 'org'}",org:구성원,wikipedia
...,...,...,...,...,...,...
447,447,"이제 올라프는 겨울 발톱 부족의 포악한 집행자가 되어, 대규모 전쟁에서 전사할 수 ...","{'start': 3, 'text': '올라프', 'type': 'per'}","{'start': 18, 'text': '포악한 집행자', 'type': 'per'}",per:직업_직함,wikipedia
448,448,다만 저그라인은 여전히 미흡한 점이 많이 지적됐다.(이날도 저그 라인은 승리를 하지...,"{'start': 40, 'text': '승리', 'type': 'com'}","{'start': 33, 'text': '저그 라인', 'type': 'org'}",관계없음,wikipedia
449,449,"발로란의 수많은 필멸자를 자신의 씨앗에서 태어난 후손의 먹잇감 정도로 여기므로, 치...","{'start': 0, 'text': '발로란', 'type': 'org'}","{'start': 9, 'text': '필멸자', 'type': 'per'}",org:구성원,wikipedia
450,450,"T1은 대한민국의 종합 프로게임단으로 리그 오브 레전드, 배틀그라운드, 포트나이트,...","{'start': 4, 'text': '대한민국의 종합 프로게임단', 'type':...","{'start': 0, 'text': 'T1', 'type': 'org'}",org:구성원,wikipedia


In [18]:
data_df.groupby("label").count()

,id,sentence,subject_entity,object_entity,source
label,,,,,
com:참가,12,12,12,12,12
org:구성원,106,106,106,106,106
org:발생,29,29,29,29,29
org:별칭,20,20,20,20,20
per:별칭,57,57,57,57,57
per:직업_직함,73,73,73,73,73
rec:달성,6,6,6,6,6
관계없음,149,149,149,149,149


In [19]:
train, test = train_test_split(data_df, test_size=0.4, shuffle=True)
test, val = train_test_split(test, test_size=0.5, shuffle=True)

In [20]:
len(train), len(test), len(val), len(data_df)

(271, 90, 91, 452)

In [78]:
# for i in range(len(test)):
#     test.iloc[i]["label"] = dict_label_to_num[test.iloc[i]["label"]]

In [22]:
train.to_csv("../dataset/train/train.csv", index=False)
val.to_csv("../dataset/train/dev.csv", index=False)
test.to_csv("../dataset/test/test_data.csv", index=False)
#data_df.to_csv("../dataset/test/test_data.csv", index=False)

## save pickle

In [23]:
set(data_df["label"].tolist())

{'com:참가',
 'org:구성원',
 'org:발생',
 'org:별칭',
 'per:별칭',
 'per:직업_직함',
 'rec:달성',
 '관계없음'}

In [24]:
dict_label_to_num = {
    '관계없음' : 0,
    'com:참가' : 1,
    'org:구성원' : 2,
    'org:발생' : 3,
    'org:별칭' : 4,
    'per:별칭' : 5,
    'per:직업_직함' : 6,
    'rec:달성' : 7                  
}
dict_num_to_label = {num:label for label, num in dict_label_to_num.items()}

In [25]:
dict_label_to_num

{'관계없음': 0,
 'com:참가': 1,
 'org:구성원': 2,
 'org:발생': 3,
 'org:별칭': 4,
 'per:별칭': 5,
 'per:직업_직함': 6,
 'rec:달성': 7}

In [26]:
dict_num_to_label

{0: '관계없음',
 1: 'com:참가',
 2: 'org:구성원',
 3: 'org:발생',
 4: 'org:별칭',
 5: 'per:별칭',
 6: 'per:직업_직함',
 7: 'rec:달성'}

In [27]:
dict_label_to_num.keys()

dict_keys(['관계없음', 'com:참가', 'org:구성원', 'org:발생', 'org:별칭', 'per:별칭', 'per:직업_직함', 'rec:달성'])

In [28]:
import pickle

# save
with open('../model/dict_label_to_num.pkl', 'wb') as f:
    pickle.dump(dict_label_to_num, f, pickle.HIGHEST_PROTOCOL)
with open('../model/dict_num_to_label.pkl', 'wb') as f:
    pickle.dump(dict_num_to_label, f, pickle.HIGHEST_PROTOCOL)

In [29]:
import pickle

# save
with open('dict_label_to_num.pkl', 'wb') as f:
    pickle.dump(dict_label_to_num, f, protocol=2)
with open('dict_num_to_label.pkl', 'wb') as f:
    pickle.dump(dict_num_to_label, f, protocol=2)